# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
name_of_cols = ['target','id','date','flag','user','text']
sent = pd.read_csv("/content/drive/MyDrive/challenges/training.1600000.processed.noemoticon.csv", sep = ',', names=name_of_cols , encoding='latin-1')
sent.head()
sent_sampled = sent.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [38]:
import re

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s=re.sub('http://.*', ' ', s).lower()
    s=re.sub('[^A-Za-z]+', ' ', s)
    
    return s

In [42]:
def tokenized(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

In [43]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer ()
    stemmed = [ps.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w) for w in stemmed]
    return lemmatized

In [44]:
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    
    stop_words = stopwords.words('english')
    
    return [w for w in l if w not in stop_words]

In [48]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

sent_sampled['text_processed'] = sent_sampled.text.apply(clean_up)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(tokenized)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(stem_and_lemmatize)
sent_sampled['text_processed'] = sent_sampled.text_processed.apply(remove_stopwords)
sent_sampled

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,target,id,date,flag,user,text,text_processed
108348,0,1824255269,Sun May 17 00:37:29 PDT 2009,NO_QUERY,silentinfinite,@FunkeeMonk What!? Wow. I'm still so behind on...,"[funkeemonk, wow, still, behind, thi, whole, r..."
1503855,4,2072053211,Sun Jun 07 20:19:14 PDT 2009,NO_QUERY,KassandraKanhai,New video by @3OH3pfr premiering tomorrow for ...,"[new, video, oh, pfr, premier, tomorrow, song,..."
1126925,4,1975105500,Sat May 30 14:27:33 PDT 2009,NO_QUERY,aaplgeek,"FYI, DreamHost.com now has $15 SSLs! U'll need...","[fyi, dreamhost, com, ha, ssl, u, need, uniqu,..."
271537,0,1990036726,Mon Jun 01 03:40:09 PDT 2009,NO_QUERY,MRCogdell11,thinking of ways to kill myself..i have worked...,"[think, way, kill, work, hard, schedul, class,..."
1012646,4,1881231037,Fri May 22 04:01:55 PDT 2009,NO_QUERY,broccolidog,"is uuwi na...have a good weekend, everyone!!! ...","[uuwi, na, good, weekend, everyon]"
...,...,...,...,...,...,...,...
996246,4,1835853667,Mon May 18 06:53:14 PDT 2009,NO_QUERY,pameladetlor,@LittleLeafsFan real short night - may as well...,"[littleleafsfan, real, short, night, may, well..."
1517603,4,2175862402,Mon Jun 15 02:12:50 PDT 2009,NO_QUERY,Leluc,@envi_envi heya m-am lasat de WoW pe timp de ...,"[envi, envi, heya, lasat, de, wow, pe, timp, d..."
1379501,4,2052139246,Fri Jun 05 23:46:47 PDT 2009,NO_QUERY,Badgirlen,@EdCullenReaLady Hey How are u today??,"[edcullenrealadi, hey, u, today]"
317718,0,2002669973,Tue Jun 02 04:51:36 PDT 2009,NO_QUERY,ConsBarcebal,@xxparanoid. Wla pa eh. Try q s iba... Hopeful...,"[xxparanoid, wla, pa, eh, tri, q, iba, hope, m..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [50]:
words = [word for line in sent_sampled.text_processed for word in line]
fdist = nltk.FreqDist(words)
top_words = [tuple[0] for tuple in fdist.most_common(5000)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [51]:
def find_features(text):
    words = text.lower()
    features = {}
    for w in top_words: features[w] = w in words
    return features

featuresets = [(find_features(t),bool(s)) for t,s in zip(sent_sampled['text'],sent_sampled['target'])]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [52]:
testing_set = featuresets[:4000]
training_set = featuresets[4000:]

classifier = nltk.NaiveBayesClassifier.train(training_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [53]:
print("Classifier accuracy percent:", nltk.classify.accuracy(classifier, testing_set)*100)

Classifier accuracy percent: 71.1
